## 행렬 요인화(MF) 기반 추천 시스템 구현

In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

In [3]:
import surprise
print(surprise.__version__)

1.1.1


In [4]:
from surprise import Dataset
from surprise.model_selection import train_test_split

# ml-100k: 10만 개 평점 데이터, 무비렌즈 사이트에서 제공하는 과거 데이터 셋 ml-100k을 불러왔다.
data = Dataset.load_builtin('ml-100k')

# surprise의 train_test_split() 사용, 10만 개의 데이터를 train 75,000, test 25,000으로 나누었다.
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [7]:
from surprise import SVD

# SVD를 이용한 잠재 요인 협업 필터링
algo = SVD()
algo.fit(trainset)

In [8]:
# 사용자 아이디(uid), 아이템 아이디(iid)는 문자열로 입력
uid = str(196)
iid = str(302)

# 추천 예측 평점 (.predict)
pred = algo.predict(uid, iid)
pred
# surprise에서 추천을 예측하는 메소드는 test()와 predict()가 있다.
# predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측하며 prediction 객체를 반환한다.
# prediction 객체는 사용자 아이디(uid), 아이템 아이디(iid), 실제 평점(r_ui), 예측 평점(est)를 튜플 형태로 가진다.
# prediction 객체의 details속성은 추천 예측을 할 수 없는 경우 True, 아닌 경우 False이다.

Prediction(uid='196', iid='302', r_ui=None, est=4.40664512877181, details={'was_impossible': False})

In [9]:
# 추천 예측 평점 (.test)
predictions = algo.test( testset )

print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')

predictions[:5]
# test()의 경우 사용자-아이템 평점 데이터 셋 전체에 대해서 추천을 예측한다.
# 반환된 리스트 객체는 25,000개의 prediction 객체를 내부에 가지고 있다.
# test()는 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행한 결과이다.

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6189169617429, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6235797360547557, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.177448313971358, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.264314206124555, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.05410191787773, details={'was_impossible': False})]

In [10]:
# 속성 확인
[ (pred.uid, pred.iid, pred.est, pred.details) for pred in predictions[:3] ]

[('120', '282', 3.6189169617429, {'was_impossible': False}),
 ('882', '291', 3.6235797360547557, {'was_impossible': False}),
 ('535', '507', 4.177448313971358, {'was_impossible': False})]

In [11]:
from surprise import accuracy

# 성능 평가
accuracy.rmse(predictions)

RMSE: 0.9481


0.9480667439638305

In [12]:
# surprise는 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 셋만 적용 가능하다.
# 만약 user_id, item_id, rating, time_stamp로 구성된 데이터가 있다면 앞 3개 컬럼만 로딩한다.
# 일반 데이터 파일이나 판다스 데이터 프레임도 로딩 가능하나 컬럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순이어야 한다.

In [13]:
# load_builtin()으로 무비렌즈 데이터를 불러올 수 있다.
# 디폴트로 ml-100k(10만 개 평점 데이터)를 불러오며 ml-1M(100만 개 평점 데이터)를 불러 올 수 있다.
# load_from_file()은 OS 파일을 로딩할 때 사용한다.
# 주의할 점은 원본 파일에 컬럼명이 없어야 한다.

In [14]:
# index와 header를 제거한 ratings_noh.csv 파일 생성
ratings = pd.read_csv('./data/ratings.csv')
ratings.to_csv('./data/ratings_noh.csv', index=False, header=False)

In [15]:
# 만약 user_id, item_id, rating, time_stamp로 구성된 데이터가 있다면 앞 3개 컬럼만 로딩한다.
# 일반 데이터 파일이나 판다스 데이터 프레임도 로딩 가능하나 컬럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순이어야 한다.

In [16]:
# load_builtin()으로 무비렌즈 데이터를 불러올 수 있다.
# 디폴트로 ml-100k(10만 개 평점 데이터)를 불러오며 ml-1M(100만 개 평점 데이터)를 불러 올 수 있다.

In [17]:
# load_from_file()은 OS 파일을 로딩할 때 사용한다.
# 주의할 점은 원본 파일에 컬럼명이 없어야 한다.
# index와 header를 제거한 ratings_noh.csv 파일 생성
ratings = pd.read_csv('./data/ratings.csv')
ratings.to_csv('./data/ratings_noh.csv', index=False, header=False)
# ratings.csv에서 index와 header를 제거한 원본 파일을 생성하였다.
# 이 데이터는 userid, movieid, rating, timestamp로 구성되어 있다.

In [18]:
from surprise import Reader

# Reader 객체 생성
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# surprise 데이터 셋은 기본적으로 무비렌즈 데이터 형식을 따른다.
# 무비렌즈 데이터 형식이 아닌 다른 OS 파일은 Reader() 클래스를 먼저 설정하여야 한다.
# line_format: 컬럼을 순서대로 나열한다(컬럼명은 공백으로 분리).
# sep: 구분자로 디폴트는 \t이다.
# rating_scale: 평점 값의 최소, 최대를 설정한다(디폴트는 1,5).
# Reader() 객체를 입력해 데이터를 불러온다.
data = Dataset.load_from_file('./data/ratings_noh.csv', reader=reader)
data 

In [19]:
# surprise의 train_test_split() 사용
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)

# 추천 예측 평점 (.test)
predictions = algo.test( testset )

# 성능 평가
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [20]:
# load_from_df()을 이용하여 데이터 프레임을 surprise 데이터 셋으로 로딩 가능하다.
# 주의할 점은 컬럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순이어야 한다.
# 데이터 불러오기 (데이터 프레임)
ratings = pd.read_csv('./data/ratings.csv') 

# Reader 객체 생성
reader = Reader(rating_scale=(0.5, 5.0))

# 사용자 아이디, 아이템 아이디, 평점 순서 (원래는 timestamp도 있으나 제외)
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# surprise의 train_test_split() 사용
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)

# 추천 예측 평점 (.test)
predictions = algo.test( testset )

# 성능 평가
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 교차 검증

In [21]:
import time
from surprise.model_selection import cross_validate 


# 데이터 불러오기 (데이터 프레임)
ratings = pd.read_csv('./data/ratings.csv') 

# Reader 객체 생성
reader = Reader(rating_scale=(0.5, 5.0))

# 사용자 아이디, 아이템 아이디, 평점 순서 (원래는 timestamp도 있으나 제외)
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# SVD를 이용한 잠재 요인 협업 필터링
algo = SVD(random_state=0)

start = time.time()
# 교차 검증 수행
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# surprise.model_selection의 cross_validate()으로 교차 검증이 가능하다.
end = time.time()
print('Runtime: {0:.2f} 초'.format(end-start))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8694  0.8768  0.8809  0.8651  0.8736  0.8732  0.0055  
MAE (testset)     0.6669  0.6736  0.6774  0.6655  0.6719  0.6710  0.0044  
Fit time          8.83    7.24    7.95    7.30    7.30    7.72    0.61    
Test time         0.28    0.21    0.36    0.18    0.20    0.25    0.07    
Runtime: 40.90 초


### 하이퍼파라미터
- GridSearchCV()로 하이퍼 파라미터 튜닝이 가능하다.

In [22]:
from surprise.model_selection import GridSearchCV

# n_epochs: SGD 수행 시 반복 횟수, n_factors: 잠재 요인 크기
param_grid = {
    'n_epochs': [20, 40, 60], 
    'n_factors': [50, 100, 200]
}

# GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3) # algo가 아닌 SVD 입력하였다.

start = time.time()
gs.fit(data)
end = time.time()
print('Runtime: {0:.2f} 초'.format(end-start))

# 최적 하이퍼 파라미터 및 그 때의 최고 성능
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

Runtime: 379.16 초
{'n_epochs': 20, 'n_factors': 50}
0.8770074214446367


In [23]:
# 오류 코드
# 실습 데이터는 ratings.csv를 train, test 분리 없이 전체로 사용한다.
# 다만 surprise는 TrainSet 객체로 변환하지 않으면 fit()으로 학습시 오류가 발생한다.
# data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# algo = SVD(n_factors=50, random_state=0)
# algo.fit(data)

In [24]:
# DatasetAutoFolds() 객체를 생성 후 build_full_trainset() 메서드를 호출하면 전체 데이터를 학습 데이터로 만들 수 있음
from surprise.dataset import DatasetAutoFolds

# Reader 객체 생성
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# DatasetAutoFolds 객체 생성
data_folds = DatasetAutoFolds(ratings_file='./data/ratings_noh.csv', reader=reader)

# 전체 데이터를 train으로 지정
trainset = data_folds.build_full_trainset()
trainset

In [25]:
# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
model = SVD(n_epochs=20, n_factors=50, random_state=0)
model.fit(trainset)  # 훈련

# 사용자 아이디, 아이템 아이디 문자열로 입력
uid = str(9)
iid = str(42)

# 추천 예측 평점 (.predict)
pred = model.predict(uid, iid, verbose=True) # 9번 회원은 42번 영화를 추천한 적이 없으며 예측 점수는 3.13점이다.

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [26]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./data/movies.csv')

# 아직 보지 않은 영화 리스트 함수
def get_unseen_surprise(ratings, movies, userId):
     # 특정 userId가 평점을 매긴 모든 영화 리스트
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    total_movies = movies['movieId'].tolist()
      
    # 한줄 for + if문으로 안 본 영화 리스트 생성
    unseen_movies = [ movie for movie in total_movies if movie not in seen_movies]
    
    # 일부 정보 출력
    total_movie_cnt = len(total_movies)
    seen_cnt = len(seen_movies)
    unseen_cnt = len(unseen_movies)
    
    print(f"전체 영화 수: {total_movie_cnt}, 평점 매긴 영화 수: {seen_cnt}, 추천 대상 영화 수: {unseen_cnt}")
    
    return unseen_movies

In [27]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)

전체 영화 수: 9742, 평점 매긴 영화 수: 46, 추천 대상 영화 수: 9696


In [28]:

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    
    # 아직 보지 않은 영화의 예측 평점: prediction 객체 생성
    predictions = []    
    for movieId in unseen_movies:
        predictions.append(algo.predict(str(userId), str(movieId)))
    
    # 리스트 내의 prediction 객체의 est를 기준으로 내림차순 정렬
    def sortkey_est(pred):
        return pred.est #est: 예측평점

    predictions.sort(key=sortkey_est, reverse=True) # key에 리스트 내 객체의 정렬 기준을 입력
    
    # 상위 top_n개의 prediction 객체
    top_predictions = predictions[:top_n]
    
    # 영화 아이디, 제목, 예측 평점 출력
    print(f"Top-{top_n} 추천 영화 리스트")
    
    for pred in top_predictions:
        
        movie_id = int(pred.iid) #영화 id
        movie_title = movies[movies["movieId"] == movie_id]["title"].tolist()
        movie_rating = pred.est #평점 예측
        
        print(f"{movie_title}: {movie_rating:.2f}") 

In [29]:
recomm_movie_by_surprise(model, 9, unseen_movies, top_n=10)

Top-10 추천 영화 리스트
['Godfather, The (1972)']: 4.31
['Star Wars: Episode IV - A New Hope (1977)']: 4.28
['Pulp Fiction (1994)']: 4.28
['Star Wars: Episode V - The Empire Strikes Back (1980)']: 4.23
['Usual Suspects, The (1995)']: 4.19
['Streetcar Named Desire, A (1951)']: 4.15
['Star Wars: Episode VI - Return of the Jedi (1983)']: 4.12
['Goodfellas (1990)']: 4.11
['Glory (1989)']: 4.08
['Silence of the Lambs, The (1991)']: 4.08


### 모델 저장및 사용

In [30]:
import pickle # 파이썬 자료형 저장 지원
import joblib # 병렬 프로그래밍, 모델 저장, 로딩

joblib.dump(model, './ml-latest-small.pkl') 

['./ml-latest-small.pkl']